# Simple Semantic Kernel Agent Tutorial

Learn to build AI agents with Semantic Kernel in just a few steps. This tutorial covers the essentials: creating agents, adding tools, and managing conversations.

There are three foundational components in Semantic Kernel agents: 
1. Agent Class: All agent types inherit from this class. Agent types include ChatCompletionAgent (uses standard chat completion APIs), OpenAIAssistantAgent (leverages OpenAI Assistant API with built in tools), AzureAIAgent (integrates with Azure AI services for enterprise scenarios), CopilotStudioAgent (connects to Microsoft Copilot Studio workflows). 
2. Agent Thread: This handles how conversation history and state are maintained. This is important since agents need context from previous messages to make informed decisions. There are two approaches:
    1. Service managed state: An agent service like Azure AI stores conversation history server-side, and accessed via thread ID.
    2. Application managed state: Your application maintains the full chat history and passes it to the agent on each call. 
3. Agent orchestration: The framework provides pre-built patterns for coordinating multiple agents to handle complex workflows that single agents cannot manage effectively. 
    1. Sequential: Agents execute one after the other in order. This is like document processing pipeline.
    2. Concurrent: Multiple agents working at the same time. Like customer inquiry handling (billing agent and account agent working in parallel).
    3. Handoff: Agents pass control to each other based on specialization. Like customer service triage to specialist agent.
    4. Group chat: Agents collaborate in a shared conversation. Like project planning with domain experts. 

Agents leverage Semantic Kernel’s plugin system to access tools, databases, etc. 

Agents can also be defined using YAML.

## Setup

Install required packages and configure your environment:

In [3]:
# Install packages
!pip install semantic-kernel python-dotenv

# Import everything we need
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

print("✅ Setup complete!")


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✅ Setup complete!


In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if API key is configured
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("⚠️  Please add OPENAI_API_KEY to your .env file!")
    print("   Create a .env file with: OPENAI_API_KEY=your_actual_key_here")
else:
    print("✅ API key configured")

✅ API key configured


## Step 1: Create a Simple Agent

Let's start with the basics - an agent that can chat. We are simply giving the kernel (our orchestrator) access to chat service, which leverages the Chat Completion API endpoint from Open AI. We will add more tools later. 

In [5]:
# Create service and kernel
chat_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    api_key=OPENAI_API_KEY
)

kernel = Kernel()
kernel.add_service(chat_service)

# Create a basic agent
agent = ChatCompletionAgent(
    kernel=kernel,
    name="Assistant",
    instructions="You are a helpful and friendly assistant."
)

# Test it
response = await agent.get_response("Hello! What can you help me with?")
print(f"🤖 {agent.name}: {response.content}")

🤖 Assistant: Hello! I'm here to assist you with a wide range of topics. I can help you with:

- Answering questions and providing information on various subjects
- Assisting with academic topics, including math, science, literature, and history
- Offering writing assistance, including editing and generating text
- Providing explanations and tutorials for specific concepts
- Helping with language learning and translation
- Giving advice on everyday topics, like cooking, technology, and more
- Engaging in casual conversation on interests or hobbies

Feel free to ask me anything specific you have in mind!


## Step 2: Add Tools (Functions)

Now let's give our agent some useful capabilities. We will give it a weather function and a calculator. Both of these are functions we define in our code and provide them to the kernel as tools. 

In [23]:
# Define useful functions
@kernel_function(description="Get current weather for a city")
def get_weather(city: str) -> str:
    """Mock weather function - replace with real API call."""
    weather_data = {
        "london": "Cloudy, 15°C",
        "paris": "Sunny, 22°C", 
        "tokyo": "Rainy, 18°C",
        "new york": "Partly cloudy, 20°C"
    }
    return weather_data.get(city.lower(), f"Weather data not available for {city}")

@kernel_function(description="Calculate simple math expressions")
def calculate(expression: str) -> str:
    """Safe calculator for basic math."""
    try:
        # Only allow basic math operations for safety
        allowed_chars = "0123456789+-*/(). "
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"{expression} = {result}"
        else:
            return "Sorry, I can only do basic math operations (+, -, *, /, parentheses)"
    except:
        return "Sorry, I couldn't calculate that. Please check your expression."

# Add functions to kernel
kernel.add_function(plugin_name="tools", function=get_weather)
kernel.add_function(plugin_name="tools", function=calculate)

# Create enhanced agent
enhanced_agent = ChatCompletionAgent(
    kernel=kernel,
    name="SmartAssistant",
    instructions="""
    You are a helpful assistant with weather and calculator capabilities.
    
    - Use get_weather when asked about weather in specific cities
    - Use calculate for math problems
    - Be friendly and explain what you're doing
    """
)

print("✅ Enhanced agent created with tools!")

✅ Enhanced agent created with tools!


## Step 3: Test the Agent with Tools

Let's see our agent use its tools:

In [24]:
# Test weather function
print("🌤️ Testing weather function:")
response = await enhanced_agent.get_response("What's the weather in London?")
print(f"🤖 {enhanced_agent.name}: {response.content}\n")

# Test calculator function  
print("🧮 Testing calculator function:")
response = await enhanced_agent.get_response("What's 25 * 4 + 10?")
print(f"🤖 {enhanced_agent.name}: {response.content}\n")

# Test general conversation
print("💬 Testing general conversation:")
response = await enhanced_agent.get_response("Tell me a fun fact about AI")
print(f"🤖 {enhanced_agent.name}: {response.content}")

🌤️ Testing weather function:
🤖 SmartAssistant: The weather in London is currently cloudy with a temperature of 15°C. If you need more details or information, feel free to ask!

🧮 Testing calculator function:
🤖 SmartAssistant: The result of \( 25 \times 4 + 10 \) is \( 110 \). If you have any more calculations or questions, feel free to ask!

💬 Testing general conversation:
🤖 SmartAssistant: Here's a fun fact about AI: The concept of artificial intelligence dates back to ancient history! The idea of creating intelligent machines can be traced to myths and stories, such as the ancient Greek myth of Talos, a giant automaton made of bronze that protected the island of Crete. Fast forward to the 1950s, and the field of AI was officially founded, with pioneers like Alan Turing and John McCarthy laying the groundwork for what we have today. Isn't it fascinating how the idea of intelligent machines has evolved over time?


## Step 4: Conversation with Memory

For conversations that remember previous messages. We will use semantic kernel out of the box memory management, but that will depend on context window.

Memory can be solved by summarizing past conversations or use a longer term memory like RAG. There is also learning memory where we want the agent to elarn from all past interactions, this is also implemented using RAG where we store successful resolution examples and retrieve them for similar cases. 

In [ ]:
async def chat_with_memory():
    """Demonstrate conversation with memory."""
    
    print("💭 Conversation with Memory Demo")
    print("=" * 40)
    
    # Messages that build on each other
    messages = [
        "Hi! I'm planning a trip to Paris.",
        "What's the weather like there?",
        "That sounds nice! Can you calculate 150 * 7 for my budget?",
        "Perfect, that should cover my week there. Thanks!"
    ]
    
    thread = None  # This will store conversation history
    
    for msg in messages:
        print(f"👤 User: {msg}")
        
        # Agent remembers previous messages through the thread
        response = await enhanced_agent.get_response(messages=msg, thread=thread)
        print(f"🤖 {enhanced_agent.name}: {response.content}\n")
        
        # Update thread to keep conversation history
        thread = response.thread

# Run the conversation demo
await chat_with_memory()

💭 Conversation with Memory Demo
👤 User: Hi! I'm planning a trip to Paris.
🤖 SmartAssistant: That sounds exciting! Paris is a beautiful city with so much to offer. Do you need help with anything specific, like weather information, places to visit, or travel tips?

👤 User: What's the weather like there?
🤖 SmartAssistant: The weather in Paris is currently sunny with a temperature of 22°C. It sounds like a lovely day to explore the city! Is there anything else you would like to know about your trip?

👤 User: That sounds nice! Can you calculate 150 * 7 for my budget?
🤖 SmartAssistant: The result of 150 multiplied by 7 is 1050. If you need help with anything else, feel free to ask!

👤 User: Perfect, that should cover my week there. Thanks!
🤖 SmartAssistant: You're very welcome! I'm glad I could help. If you have any more questions or need assistance with anything else as you prepare for your trip to Paris, just let me know. Have an amazing time! 🌍✈️



## Step 5: See Tools in Action (Advanced)

Watch exactly what happens when your agent uses tools:

In [26]:
async def show_tool_usage():
    """Show detailed tool execution."""
    
    print("🔧 Tool Usage Demo")
    print("=" * 25)
    
    # Callback to see tool calls
    async def log_tool_calls(message):
        from semantic_kernel.contents import FunctionCallContent, FunctionResultContent
        
        for item in message.items or []:
            if isinstance(item, FunctionCallContent):
                print(f"  🔧 Calling: {item.name}({item.arguments})")
            elif isinstance(item, FunctionResultContent):
                print(f"  ✅ Result: {item.result}")
    
    user_input = "What's the weather in Tokyo and what's 15 + 27?"
    print(f"👤 User: {user_input}\n")
    
    # Use invoke to see intermediate steps
    async for response in enhanced_agent.invoke(
        messages=user_input,
        on_intermediate_message=log_tool_calls
    ):
        print(f"\n🤖 Final Response: {response.content}")

# Run the tool usage demo
await show_tool_usage()

🔧 Tool Usage Demo
👤 User: What's the weather in Tokyo and what's 15 + 27?

  🔧 Calling: tools-get_weather({"city": "Tokyo"})
  🔧 Calling: tools-calculate({"expression": "15 + 27"})
  ✅ Result: Rainy, 18°C
  ✅ Result: 15 + 27 = 42

🤖 Final Response: The weather in Tokyo is currently rainy, with a temperature of 18°C. 

As for your math question, 15 + 27 equals 42. 

If you need anything else, just let me know!


## Step 6: Streaming Responses

For real-time responses (like ChatGPT):

In [27]:
async def streaming_demo():
    """Show streaming responses."""
    
    print("\n🌊 Streaming Demo")
    print("=" * 20)
    
    print("👤 User: Write a short poem about coding")
    print("🤖 Assistant: ", end="", flush=True)
    
    # Stream response word by word
    async for chunk in enhanced_agent.invoke_stream(
        messages="Write a short poem about coding"
    ):
        print(chunk.content, end="", flush=True)
    
    print("\n")  # New line when done

# Run streaming demo
await streaming_demo()


🌊 Streaming Demo
👤 User: Write a short poem about coding
🤖 Assistant: In lines of code, a story's spun,  
Logic dances, algorithms run.  
With every loop, a world unfolds,  
In syntax rich, creativity molds.  

From bugs that hide in shadows deep,  
To functions crafted, secrets to keep.  
In the glow of screens, ideas ignite,  
Coding's the magic, day or night.  

With every keystroke, dreams take flight,  
Building the future, pixel by light.  
So here’s to the coders, both near and far,  
Crafting new realms, like a digital star!




## Summary: What You've Learned

In [ ]:
print("🎓 What You've Built:")
print("=" * 30)

summary = [
    "✅ Basic AI agent with OpenAI",
    "✅ Custom tools/functions for weather and math", 
    "✅ Conversation memory management",
    "✅ Tool execution monitoring",
    "✅ Real-time streaming responses"
]

for item in summary:
    print(item)

print("\n🚀 Key Concepts:")
concepts = {
    "Agent": "AI that can reason, remember, and use tools",
    "Kernel": "Manages AI services and functions",
    "Functions": "Tools that extend agent capabilities", 
    "Thread": "Maintains conversation history",
    "Streaming": "Real-time response generation"
}

for concept, description in concepts.items():
    print(f"• {concept}: {description}")

print("\n💡 Next Steps:")
print("• Try different models (gpt-4, gpt-3.5-turbo)")
print("• Create custom functions for your use case")
print("• Explore multi-agent conversations")
print("• Add guardrails for production safety")

## Step 7: Sequential Agent Orchestration

Now let's see how multiple agents can work together in a pipeline - each agent processes the output from the previous one. Notice how easy Semantic Kernel makes this for us:

In [28]:
# Import orchestration components
from semantic_kernel.agents import SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import ChatMessageContent

print("🔗 Setting up Sequential Agent Pipeline")
print("=" * 45)

🔗 Setting up Sequential Agent Pipeline


In [30]:
# Create specialized agents for a marketing pipeline
def create_marketing_pipeline():
    """Create three agents that work together sequentially."""
    
    # Agent 1: Extract key information
    concept_extractor = ChatCompletionAgent(
        name="ConceptExtractor",
        instructions="""
        You are a marketing analyst. Given a product description, identify:
        - Key features (bullet points)
        - Target audience 
        - Unique selling points
        
        Format your output clearly with headers.
        """,
        kernel=kernel
    )
    
    # Agent 2: Write marketing copy
    copywriter = ChatCompletionAgent(
        name="Copywriter", 
        instructions="""
        You are a marketing copywriter. Take the analysis provided and write 
        compelling marketing copy (around 100-150 words). Make it engaging 
        and highlight the key benefits. Output just the marketing copy.
        """,
        kernel=kernel
    )
    
    # Agent 3: Polish and format
    editor = ChatCompletionAgent(
        name="Editor",
        instructions="""
        You are an editor. Take the marketing copy and polish it:
        - Fix grammar and clarity
        - Ensure consistent tone
        - Make it more compelling
        - Output the final polished version
        """,
        kernel=kernel
    )
    
    return [concept_extractor, copywriter, editor]

# Create the pipeline
marketing_agents = create_marketing_pipeline()
print(f"✅ Created {len(marketing_agents)} specialized agents:")
for agent in marketing_agents:
    print(f"   • {agent.name}")

✅ Created 3 specialized agents:
   • ConceptExtractor
   • Copywriter
   • Editor


In [31]:
# Set up callback to see each agent's work
def agent_callback(message: ChatMessageContent) -> None:
    """Show what each agent produces."""
    print(f"\n🤖 {message.name}:")
    print("-" * 30)
    print(message.content)
    print()

# Create the sequential orchestration
sequential_pipeline = SequentialOrchestration(
    members=marketing_agents,
    agent_response_callback=agent_callback
)

print("🔗 Sequential pipeline created!")

🔗 Sequential pipeline created!


In [32]:
# Run the sequential pipeline
async def run_marketing_pipeline():
    """Execute the sequential agent pipeline."""
    
    print("🚀 Running Marketing Pipeline")
    print("=" * 35)
    
    # Start the runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        # Input: Product description
        product_description = (
            "A smart water bottle with temperature display, "
            "app connectivity, hydration reminders, and "
            "leak-proof design. Made from BPA-free materials."
        )
        
        print(f"📝 Input Product: {product_description}\n")
        print("Processing through pipeline...")
        
        # Run the sequential orchestration
        result = await sequential_pipeline.invoke(
            task=product_description,
            runtime=runtime
        )
        
        # Get final result
        final_output = await result.get(timeout=30)
        
        print("🎯 FINAL MARKETING COPY:")
        print("=" * 40)
        print(final_output)
        
    finally:
        # Clean up
        await runtime.stop_when_idle()

# Execute the pipeline
await run_marketing_pipeline()

🚀 Running Marketing Pipeline
📝 Input Product: A smart water bottle with temperature display, app connectivity, hydration reminders, and leak-proof design. Made from BPA-free materials.

Processing through pipeline...

🤖 ConceptExtractor:
------------------------------
## Key Features
- Temperature display for accurate monitoring
- App connectivity for tracking hydration levels
- Hydration reminders to encourage regular drinking
- Leak-proof design to prevent spills
- Made from BPA-free materials for safety and health

## Target Audience
- Health-conscious individuals
- Fitness enthusiasts and athletes
- Busy professionals needing hydration reminders
- Eco-friendly consumers looking for sustainable products

## Unique Selling Points
- Combines technology with convenience to promote healthy hydration habits
- Customizable app features provide personalized hydration tracking
- Durable and leak-proof construction for on-the-go use
- Safe, BPA-free materials appeal to health-conscious users

In [33]:
# Quick demo with a different product
async def quick_pipeline_demo():
    """Quick demo with another product."""
    
    print("\n🔄 Quick Pipeline Demo #2")
    print("=" * 30)
    
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        # Different product
        product = "Wireless earbuds with 30-hour battery, noise cancellation, and workout-proof design"
        
        print(f"📝 Product: {product}")
        
        # Simple pipeline without detailed logging
        simple_pipeline = SequentialOrchestration(members=marketing_agents)
        
        result = await simple_pipeline.invoke(
            task=product,
            runtime=runtime
        )
        
        final_copy = await result.get(timeout=30)
        print(f"\n🎯 Final Copy:\n{final_copy}")
        
    finally:
        await runtime.stop_when_idle()

# Run quick demo
await quick_pipeline_demo()


🔄 Quick Pipeline Demo #2
📝 Product: Wireless earbuds with 30-hour battery, noise cancellation, and workout-proof design

🎯 Final Copy:
Elevate your audio experience with our state-of-the-art earbuds, meticulously crafted for the modern lifestyle. Enjoy an impressive 30-hour battery life, allowing you to immerse yourself in your favorite tunes from dawn till dusk, free from the hassle of constant recharging. With advanced noise cancellation technology, you can say goodbye to distractions and embrace unparalleled sound quality, whether on your daily commute or flying across the country.

Designed with fitness enthusiasts in mind, our workout-proof earbuds offer sweat and water resistance, enabling you to push your limits without worry. Experience the perfect blend of durability and performance as they stay securely in place, even during the most intense workouts. Discover the earbuds that keep pace with your active life—because your music deserves to be as dynamic as you are!



## Summary: What You've Learned

In [ ]:
print("🎓 Complete Tutorial Summary:")
print("=" * 35)

summary = [
    "✅ Basic AI agent with OpenAI",
    "✅ Custom tools/functions for weather and math", 
    "✅ Conversation memory management",
    "✅ Tool execution monitoring",
    "✅ Real-time streaming responses",
    "✅ Sequential agent orchestration"
]

for item in summary:
    print(item)

print("\n🚀 Key Concepts:")
concepts = {
    "Agent": "AI that can reason, remember, and use tools",
    "Kernel": "Manages AI services and functions",
    "Functions": "Tools that extend agent capabilities", 
    "Thread": "Maintains conversation history",
    "Streaming": "Real-time response generation",
    "Sequential Orchestration": "Pipeline where agents process output sequentially"
}

for concept, description in concepts.items():
    print(f"• {concept}: {description}")

print("\n💡 Agent Patterns:")
patterns = [
    "Single Agent: One agent handles entire workflow",
    "Sequential: Agents work in pipeline (A → B → C)",
    "Concurrent: Multiple agents work simultaneously", 
    "Manager: Central agent coordinates specialists",
    "Handoff: Agents pass control to each other"
]

for pattern in patterns:
    print(f"• {pattern}")

print("\n🎯 When to Use Sequential Orchestration:")
use_cases = [
    "• Document processing (extract → summarize → format)",
    "• Content creation (research → write → edit)", 
    "• Data analysis (collect → analyze → visualize)",
    "• Code review (analyze → suggest → validate)"
]

for use_case in use_cases:
    print(use_case)

---

**🔑 Key Takeaways:**
- **Single Agents**: Great for simple tasks and learning
- **Sequential Orchestration**: Perfect for multi-step workflows where each step builds on the previous
- **Specialization**: Each agent focuses on what it does best
- **Pipeline Benefits**: Better quality through specialized processing
- **Real-world Applications**: Document processing, content creation, data analysis

This tutorial covers everything from basic agents to sophisticated multi-agent pipelines!